In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook

In [2]:
n = str(2) #chiller number
datatypes = ["Power","ConFlow","EvaFlow","Temp"]
df={}
for datatype in datatypes:
    df[datatype] = pd.read_csv("data/Chiller"+n+"_"+datatype+"_full.csv")
    df[datatype]['ts']=pd.to_datetime(df[datatype]['ts'])
    df[datatype]['ts']=df[datatype]['ts'].dt.floor('min')
    print(datatype)
    print(df[datatype].columns)

Power
Index([u'ts', u'gatewayId', u'linkQuality', u'ch1Watt', u'ch2Watt', u'ch3Watt',
       u'totalPositiveWattHour', u'totalNegativeWattHour', u'ch1Current',
       u'ch2Current', u'ch3Current', u'ch1Voltage', u'ch2Voltage',
       u'ch3Voltage', u'ch1PowerFactor', u'ch2PowerFactor', u'ch3PowerFactor',
       u'voltage12', u'voltage23', u'voltage31', u'ch1Hz', u'ch2Hz', u'ch3Hz',
       u'i1THD', u'i2THD', u'i3THD', u'v1THD', u'v2THD', u'v3THD'],
      dtype='object')
ConFlow
Index([u'ts', u'gatewayId', u'linkQuality', u'flowRate', u'flowSpeed',
       u'totalFlowRate', u'positiveTotalFlow', u'positiveTotalFlowDecimal'],
      dtype='object')
EvaFlow
Index([u'ts', u'gatewayId', u'linkQuality', u'flowRate', u'flowSpeed',
       u'totalFlowRate', u'positiveTotalFlow', u'positiveTotalFlowDecimal'],
      dtype='object')
Temp
Index([u'ts', u'gatewayId', u'linkQuality', u'ain1', u'ain2', u'ain3', u'ain4',
       u'ain5', u'voltage1', u'voltage2', u'voltage3', u'voltage4',
       u'voltage

In [6]:
collist={}
collist['Power'] = ['ch'+str(i+1)+'Watt' for i in range(3)]
collist['Temp'] = ['value'+str(i+1) for i in range(4)]
collist['ConFlow'] = ['flowRate', 'flowSpeed']
collist['EvaFlow'] = ['flowRate', 'flowSpeed']

truncate_df = {}
for key in df.keys():
    truncate_df[key] = (df[key][['ts']+collist[key]])

truncate_df['ConFlow'].columns = ['ts', 'conflowRate', 'conflowSpeed']
truncate_df['EvaFlow'].columns = ['ts', 'evaflowRate', 'evaflowSpeed']
PowerTemp = pd.DataFrame.merge(truncate_df['Power'], truncate_df['Temp'])

Flows = pd.DataFrame.merge(truncate_df['ConFlow'], truncate_df['EvaFlow'])
fullDF = pd.DataFrame.merge(PowerTemp,Flows)
fullDF.set_index('ts')
# fullDF.index=fullDF.ts
# fullDF.drop('ts', axis=1, inplace=True)
fullDF.head()

,ts,ch1Watt,ch2Watt,ch3Watt,value1,value2,value3,value4,conflowRate,conflowSpeed,evaflowRate,evaflowSpeed
0,2017-05-01 00:00:00,325,0,324,18.580934,21.582337,23.633118,23.451539,0.0,0.0,0.0,0.0
1,2017-05-01 00:01:00,326,0,328,18.602295,21.539612,23.558351,23.398133,0.0,0.0,0.0,0.0
2,2017-05-01 00:02:00,327,0,324,18.495484,21.614379,23.483581,23.323363,0.0,0.0,0.0,0.0
3,2017-05-01 00:03:00,338,0,325,18.474123,21.571657,23.601076,23.205871,0.0,0.0,0.0,0.0
4,2017-05-01 00:04:00,334,0,328,18.559570,21.560973,23.536987,23.227236,0.0,0.0,0.0,0.0


In [7]:
fullDF.to_csv("data/Chiller"+n+"_full.csv")

In [20]:
hour = pd.to_datetime(fullDF.ts).dt.hour

In [21]:
hour

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
          ..
430540    22
430541    22
430542    22
430543    22
430544    22
430545    22
430546    22
430547    23
430548    23
430549    23
430550    23
430551    23
430552    23
430553    23
430554    23
430555    23
430556    23
430557    23
430558    23
430559    23
430560    23
430561    23
430562    23
430563    23
430564    23
430565    23
430566    23
430567    23
430568    23
430569    23
Name: ts, Length: 430570, dtype: int64